In [ ]:
import numpy as np
import pandas as pd
import os.path

root_path = os.path.dirname(os.getcwd())

# Load business license data
licenses = pd.read_csv(os.path.join(root_path, "DATA/business_licenses.csv"))

# Load food inspection data
inspections = pd.read_csv(os.path.join(root_path, "DATA/food_inspections.csv"))

# Load violation matrix
violation_matrix = pd.read_csv(os.path.join(root_path, "DATA/violation_matrix.csv"))

### CALCULATE FEATURES BASED ON FOOD INSPECTION DATA

In [36]:
# Create basis for model_data (risk not included in Chicago repository)
data = inspections[["inspection_date", "license", "inspection_id", "facility_type", "results", "risk"]]

In [38]:
# Merge with violation data
data = pd.merge(data, violation_matrix, on="inspection_id")

In [39]:
# Create pass / fail flags
pass_flags = data.results.map(lambda x: 1 if x == "Pass" else 0)
fail_flags = data.results.map(lambda x: 1 if x == "Fail" else 0)
data["pass_flags"], data["fail_flags"] = pass_flags, fail_flags

In [46]:
# Sort inspections by date
data.sort_values(by="inspection_date", inplace=True)

# Find previous inspection by shifting columns (grouped by license)
data["past_inspection_id"] = data.groupby("license").inspection_id.shift(1)


In [49]:
# Find previous violation data
thing = pd.merge(data, violation_matrix, left_on="past_inspection_id", right_on="inspection_id", suffixes=["", "_prev"])

In [52]:
past_columns = ["past_" + name for name in violation_matrix.columns]

In [53]:
past_columns

['past_v_1',
 'past_v_2',
 'past_v_3',
 'past_v_4',
 'past_v_5',
 'past_v_6',
 'past_v_7',
 'past_v_8',
 'past_v_9',
 'past_v_10',
 'past_v_11',
 'past_v_12',
 'past_v_13',
 'past_v_14',
 'past_v_15',
 'past_v_16',
 'past_v_17',
 'past_v_18',
 'past_v_19',
 'past_v_20',
 'past_v_21',
 'past_v_22',
 'past_v_23',
 'past_v_24',
 'past_v_25',
 'past_v_26',
 'past_v_27',
 'past_v_28',
 'past_v_29',
 'past_v_30',
 'past_v_31',
 'past_v_32',
 'past_v_33',
 'past_v_34',
 'past_v_35',
 'past_v_36',
 'past_v_37',
 'past_v_38',
 'past_v_39',
 'past_v_40',
 'past_v_41',
 'past_v_42',
 'past_v_43',
 'past_v_44',
 'past_v_45',
 'past_v_70',
 'past_critical_count',
 'past_serious_count',
 'past_minor_count',
 'past_inspection_id']